# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

from numpy.random import RandomState
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#читаем файл с сайта или с компа 
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df0 = pd.read_csv('C:/Users/Dmitrii/Documents/Dstudy/Skvagina/geo_data_0.csv')
    df1 = pd.read_csv('C:/Users/Dmitrii/Documents/Dstudy/Skvagina/geo_data_1.csv')
    df2 = pd.read_csv('C:/Users/Dmitrii/Documents/Dstudy/Skvagina/geo_data_2.csv')

In [3]:
#создание списка фреймов для цикла
frms = [df0,df1,df2]
for i in range(0,3):
    print('\n---------скважина',i+1,'---------')
    frms[i].info()


---------скважина 1 ---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

---------скважина 2 ---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

---------скважина 3 ---------
<class 'pandas.core.frame.DataFrame'>
R

Для анализа предоставлены три файла c данными по скважинам разных регионов. Каждый файл представляет собой 100000 строчек данных без пропусков, что идеально подходит для анализа(данные синтетические, поэтому удивительного ничего нет). 

In [4]:
for i in range(0,3):
    print('\n----------------скважина',i+1,'---------------')
    print(frms[i].head(3))



----------------скважина 1 ---------------
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647

----------------скважина 2 ---------------
      id         f0        f1        f2     product
0  kBEdx -15.001348 -8.276000 -0.005876    3.179103
1  62mP7  14.272088 -3.475083  0.999183   26.953261
2  vyE1P   6.263187 -5.948386  5.001160  134.766305

----------------скважина 3 ---------------
      id        f0        f1        f2    product
0  fwXo0 -1.146987  0.963328 -0.828965  27.758673
1  WJtFt  0.262778  0.269839 -2.530187  56.069697
2  ovLUW  0.194587  0.289035 -5.586433  62.871910


## Обучение и проверка модели

Для анализа объединим три датафрейма в один список для удобства обработки в последующем с помощью циклов. Целевой признак для обучения модели - количество запасов нефти в скважине. Столбец с названием скважины не используется для обучения и удален из данных. 

In [5]:
features = []
target = []
#Разделение на выборку с признаками и целевую
for i in range(3):
    features.append(frms[i].drop(['id','product'], axis=1))
    target.append(frms[i]['product'])

In [6]:
#Разделение на выборку тренировочную и целевую
features_train = []
features_valid = []
target_train = []
target_valid = []
for i in range(3):
    ft_trn, ft_val, ta_trn,ta_val = train_test_split(
        features[i],
        target[i], 
        test_size=0.25, 
        random_state=12345)
    features_train.append(ft_trn)
    features_valid.append(ft_val)
    target_train.append(ta_trn)
    target_valid.append(ta_val)   
    
#обучение модели и её оценка 
predictions_valid = []

for i in range(3):    
    scaler = StandardScaler()
    features_train[i] = scaler.fit_transform(features_train[i])
    features_valid[i] = scaler.transform(features_valid[i])
    
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])
    predictions_valid.append(  pd.Series(model.predict(features_valid[i])))
    rmse = (mean_squared_error(target_valid[i],predictions_valid[i]))**(0.5)
    mean_product = sum(predictions_valid[i]) / len(predictions_valid[i])
    target_valid[i] = target_valid[i].reset_index(drop=True)
    
    print("RMSE модели линейной регрессии на валидационной выборке регион",i+1,":", rmse)
    print("Средний запас предсказанного сырья в регионе",i+1,":", mean_product)

RMSE модели линейной регрессии на валидационной выборке регион 1 : 37.5794217150813
Средний запас предсказанного сырья в регионе 1 : 92.59256778438008
RMSE модели линейной регрессии на валидационной выборке регион 2 : 0.893099286775617
Средний запас предсказанного сырья в регионе 2 : 68.7285468954458
RMSE модели линейной регрессии на валидационной выборке регион 3 : 40.02970873393434
Средний запас предсказанного сырья в регионе 3 : 94.96504596800506


## Вывод
Лучшие результаты показала модель во втором регионе. При этом в 1-ом и 3-ем регионе лучшие средние величины запасов нефти. Однако это не значит, что при разработке ограниченного количества скважин лучшим для бурения будут скважины 1-го и 3-го региона.  

## Подготовка к расчёту прибыли

Запишем в константы бизнес-параметры. Посчитаем минимальный запас в скважине нефти исходя из её стоимости и разведки. 

In [7]:
RESEARCH_POINT = 200 # Количество точек при разведке
BUDGET = 10000000000 # Бюджет на разработку
DRILLING_COST = BUDGET/RESEARCH_POINT # Средняя стоимость бурения одной скважины
ORDER_COST = 450000 # Прибыль с одной единицы.
print('Минимальный средний объём сырья равен', DRILLING_COST / ORDER_COST, 'единиц')

Минимальный средний объём сырья равен 111.11111111111111 единиц


Напишем функцию, которая будет вычислять прибыльность. 

In [8]:
def profit(target, predictions):
    pred_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[pred_sorted.index][:RESEARCH_POINT]
    revenue = ORDER_COST * (selected.sum())
    return (revenue - BUDGET)

## Вывод
Учитывая, что минимальный объем сырья достаточный для разработки равен примерно 111 тыс. единиц,а средний объем запасов равен 69 тыс. единиц, то все три региона не подходят для разработки. Однако если разрабатывать только самые лучшие скважины, то можно получить прибыль.

## Расчёт прибыли и рисков 
Напишем функцию, которая работает по методу Bootstrap. На вход она получает целевой признак "target_valid" и предсказание модели "predictions" А на выходе выводит значения средней прибыли, 95% интервал дохода, вероятность получения убытоков. 


Выведем для каждого региона эти параметры.

In [9]:
state = RandomState(12345)

def bootstrap(target_valid, predictions):
    values = []
    for i in range(1000):
        target_subsample = target_valid.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
    values = pd.Series(values)
    conf_int_low = values.quantile(q=0.025)
    conf_int_high = values.quantile(q=0.975)
    risk = len(values[values < 0]) / len(values)
        
    print('Средняя прибыль в регионе = {:.2f} млрд'. format(values.mean()))
    print('95%-й доверительный интервал находится в промежутке между {:.2f} млрд и {:.2f} млрд руб.'.format(conf_int_low,conf_int_high))
    print('Риск убытков равен {:.1%}'.format(risk))

for i in range(0,3):
    print('-----Регион',i+1,'------')
    bootstrap(target_valid[i], predictions_valid[i])

-----Регион 1 ------
Средняя прибыль в регионе = 425938526.91 млрд
95%-й доверительный интервал находится в промежутке между -102090094.84 млрд и 947976353.36 млрд руб.
Риск убытков равен 6.0%
-----Регион 2 ------
Средняя прибыль в регионе = 518259493.70 млрд
95%-й доверительный интервал находится в промежутке между 128123231.43 млрд и 953612982.07 млрд руб.
Риск убытков равен 0.3%
-----Регион 3 ------
Средняя прибыль в регионе = 420194005.34 млрд
95%-й доверительный интервал находится в промежутке между -115852609.16 млрд и 989629939.84 млрд руб.
Риск убытков равен 6.2%


## Вывод
Риски убытков во всех трех регионах различны. Наибольший риск для первого и третьего региона. Риск для бурения во втором регионе минимален. 

Самым перспективым является второй регион, потому что у него наименьший риск вложений. Модель предсказания запасов для этого региона имеет наименьшую погрешность. Однако остальные регионы лишь хуже для разработки.